<a href="https://colab.research.google.com/github/minhaj-mhd/two-tower-recommedation/blob/main/two_tower_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("⏳ Installing and upgrading all required packages...")

%pip install --upgrade -q tensorflow tensorflow-recommenders tf-keras tensorflow-text
%pip install -q faiss-cpu

print("\n✅ All packages have been installed and upgraded.")

⏳ Installing and upgrading all required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.5 MB/s eta 0:00:00

✅ All packages have been installed and upgraded.


In [ ]:
%pip install --upgrade -q tensorflow-decision-forests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 92.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tf_keras
import faiss
import tensorflow_text as tf_text
import tensorflow_decision_forests as tfdf

print(f"tensorflow: {tf.__version__}")
print(f"tensorflow-recommenders: {tfrs.__version__}")
print(f"tf-keras: {tf_keras.__version__}")
print(f"faiss-cpu: {faiss.__version__}")
print(f"tensorflow-text: {tf_text.__version__}")
print(f"tensorflow-decision-forests: {tfdf.__version__}")

ImportError: cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/dtypes.py)

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.layers import TextVectorization
import faiss
from collections import defaultdict

# --- Step 1: Fabricate Data with 20 Categories and Diverse User Behavior ---
print("[1] Fabricating data with 20 categories and diverse user preferences...")

# 20 diverse categories
categories = [
    "electronics", "clothing", "books", "home_garden", "sports_outdoors",
    "beauty_health", "automotive", "toys_games", "jewelry", "music",
    "movies_tv", "kitchen_dining", "office_supplies", "pet_supplies", "crafts",
    "industrial", "grocery", "baby_products", "shoes", "watches"
]

num_items = 10000  # Increased items to accommodate more categories
num_users = 1000   # Increased users for better diversity

# Create items with explicit category tracking
item_titles = [f"Product {i}" for i in range(num_items)]
item_categories = [categories[i % len(categories)] for i in range(num_items)]

# Create diverse descriptions based on category
description_templates = {
    "electronics": [
        "High-tech electronic device with advanced features",
        "Smart gadget with wireless connectivity and AI integration",
        "Innovative electronic tool for modern digital life",
        "Premium electronic device with cutting-edge technology"
    ],
    "clothing": [
        "Fashionable apparel made from quality sustainable materials",
        "Comfortable and stylish garment for everyday wear",
        "Trendy clothing with modern design and premium fabric",
        "Versatile wardrobe piece suitable for various occasions"
    ],
    "books": [
        "Educational book covering important academic topics",
        "Engaging literature for avid readers and book enthusiasts",
        "Informative guide with practical knowledge and insights",
        "Bestselling book with compelling storytelling and research"
    ],
    "home_garden": [
        "Durable home improvement tool for DIY projects",
        "Garden equipment for landscaping and plant care",
        "Home decor item to enhance living space aesthetics",
        "Functional household item for daily convenience"
    ],
    "sports_outdoors": [
        "Professional sports equipment for athletic performance",
        "Outdoor gear for adventure and recreational activities",
        "Fitness equipment for home workout routines",
        "Camping and hiking essentials for outdoor enthusiasts"
    ],
    "beauty_health": [
        "Premium skincare product with natural ingredients",
        "Health supplement for wellness and vitality",
        "Cosmetic item for beauty enhancement and self-care",
        "Personal care product for daily hygiene routine"
    ],
    "automotive": [
        "High-quality automotive part for vehicle maintenance",
        "Car accessory for enhanced driving experience",
        "Professional-grade tool for automotive repair",
        "Vehicle enhancement product for performance optimization"
    ],
    "toys_games": [
        "Educational toy for children's development and learning",
        "Board game for family entertainment and bonding",
        "Creative plaything that sparks imagination and creativity",
        "Interactive game for skill development and fun"
    ],
    "jewelry": [
        "Elegant jewelry piece crafted with precious metals",
        "Stylish accessory for fashion and personal expression",
        "Handcrafted jewelry with unique design elements",
        "Luxury jewelry item for special occasions"
    ],
    "music": [
        "Professional music equipment for audio production",
        "Musical instrument for creative expression and performance",
        "High-quality audio device for music enthusiasts",
        "Music accessory for enhanced listening experience"
    ],
    "movies_tv": [
        "Entertainment media for leisure and relaxation",
        "Classic film collection for movie enthusiasts",
        "TV series with compelling storylines and characters",
        "Documentary content for educational entertainment"
    ],
    "kitchen_dining": [
        "Professional kitchen utensil for culinary excellence",
        "Dining accessory for elegant meal presentation",
        "Cooking tool made from premium food-safe materials",
        "Kitchen gadget for efficient food preparation"
    ],
    "office_supplies": [
        "Professional office equipment for workplace productivity",
        "Stationery item for organization and documentation",
        "Ergonomic office accessory for comfort and efficiency",
        "Business tool for professional operations"
    ],
    "pet_supplies": [
        "Pet care product for animal health and happiness",
        "Pet toy for entertainment and exercise",
        "Pet accessory for comfort and safety",
        "Pet nutrition product for optimal health"
    ],
    "crafts": [
        "Art supply for creative projects and expression",
        "Craft material for DIY projects and hobbies",
        "Creative tool for artistic endeavors and crafting",
        "Handcraft supply for personalized creations"
    ],
    "industrial": [
        "Industrial equipment for manufacturing and production",
        "Heavy-duty tool for professional industrial use",
        "Machinery component for industrial operations",
        "Professional-grade equipment for industrial applications"
    ],
    "grocery": [
        "Premium food product for nutritious meals",
        "Organic ingredient for healthy cooking",
        "Gourmet food item for culinary excellence",
        "Essential grocery item for daily nutrition"
    ],
    "baby_products": [
        "Safe baby product for infant care and development",
        "Baby accessory for comfort and convenience",
        "Child safety item for protection and security",
        "Developmental toy for early childhood learning"
    ],
    "shoes": [
        "Comfortable footwear for daily wear and activities",
        "Athletic shoe for sports and fitness activities",
        "Fashion shoe for style and personal expression",
        "Professional footwear for workplace requirements"
    ],
    "watches": [
        "Precision timepiece with advanced features",
        "Luxury watch for style and status",
        "Sports watch for active lifestyle tracking",
        "Smart watch with digital connectivity features"
    ]
}

item_descriptions = []
for i in range(num_items):
    category = item_categories[i]
    template = description_templates[category][i % len(description_templates[category])]
    item_descriptions.append(f"{template}. Model v{i % 15}. Item #{i}.")

items_data = {
    "item_id": [str(i) for i in range(num_items)],
    "item_title": item_titles,
    "item_description": item_descriptions,
    "category": item_categories
}
items_df = pd.DataFrame(items_data)

# Create category-to-items mapping for easier lookup
category_to_items = defaultdict(list)
for idx, row in items_df.iterrows():
    category_to_items[row['category']].append(row['item_id'])

# Generate user interactions: each user interacts with 3-5 categories, 1-5 interactions per category
print("Generating user interactions with diverse category preferences...")
user_interactions = []
user_categories = {}  # Track which categories each user prefers

for user_id in range(num_users):
    # Each user randomly selects 3-5 categories
    num_categories = np.random.randint(3, 6)  # 3 to 5 categories
    preferred_categories = np.random.choice(categories, size=num_categories, replace=False)
    user_categories[str(user_id)] = preferred_categories

    # Generate interactions for each preferred category
    for category in preferred_categories:
        # Random number of interactions per category (1-5)
        num_interactions = np.random.randint(1, 6)  # 1 to 5 interactions

        # Select random items from this category
        available_items = category_to_items[category]
        selected_items = np.random.choice(available_items, size=num_interactions, replace=True)

        for item_id in selected_items:
            user_interactions.append({
                "user_id": str(user_id),
                "item_id": item_id
            })

interactions_df = pd.DataFrame(user_interactions)

# Calculate statistics
total_interactions = len(interactions_df)
avg_interactions_per_user = total_interactions / num_users
category_interaction_counts = defaultdict(int)
user_category_counts = defaultdict(int)

for _, interaction in interactions_df.iterrows():
    item_category = items_df[items_df['item_id'] == interaction['item_id']]['category'].iloc[0]
    category_interaction_counts[item_category] += 1

for user_id, cats in user_categories.items():
    user_category_counts[len(cats)] += 1

print(f"Generated {len(items_df)} items across {len(categories)} categories")
print(f"Generated {total_interactions} interactions from {num_users} users")
print(f"Average interactions per user: {avg_interactions_per_user:.1f}")
print(f"\nUser category distribution:")
for num_cats, count in sorted(user_category_counts.items()):
    print(f"  {num_cats} categories: {count} users ({count/num_users:.1%})")

print(f"\nTop 10 most interacted categories:")
sorted_categories = sorted(category_interaction_counts.items(), key=lambda x: x[1], reverse=True)
for i, (cat, count) in enumerate(sorted_categories[:10]):
    print(f"  {i+1}. {cat}: {count} interactions")

# Display sample user preferences
print("\nSample user category preferences:")
for i in range(10):
    user_id = str(i)
    cats = user_categories[user_id]
    user_interactions_count = len(interactions_df[interactions_df['user_id'] == user_id])
    print(f"User {user_id}: {list(cats)} ({user_interactions_count} total interactions)")

items_ds = tf.data.Dataset.from_tensor_slices(dict(items_df))

# --- Step 2: Enhanced Item Tower with Category Integration ---
print("\n[2] Building and training the enhanced Item Tower with category integration...")
embedding_dimension = 128
max_tokens = 15_000
sequence_length = 120

# Create text vectorizers for both description and category
description_vectorizer = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=sequence_length,
    name="description_vectorizer"
)
description_vectorizer.adapt(items_ds.map(lambda x: x["item_description"]).batch(256))

# Create category vectorizer (much smaller vocabulary)
category_vectorizer = TextVectorization(
    max_tokens=len(categories) + 10,  # Small vocabulary for categories
    output_sequence_length=1,  # Categories are single tokens
    name="category_vectorizer"
)
category_vectorizer.adapt(items_ds.map(lambda x: x["category"]).batch(256))

class EnhancedItemModel(tf.keras.Model):
    def __init__(self, description_vectorizer, category_vectorizer, embedding_dim=128):
        super().__init__()
        self.description_vectorizer = description_vectorizer
        self.category_vectorizer = category_vectorizer
        self.embedding_dim = embedding_dim

        # Description embedding branch
        self.description_embedding = tf.keras.Sequential([
            self.description_vectorizer,
            tf.keras.layers.Embedding(
                input_dim=self.description_vectorizer.vocabulary_size(),
                output_dim=embedding_dim,
                mask_zero=True,
                name="description_embedding"
            ),
            tf.keras.layers.GlobalAveragePooling1D(name="description_pooling"),
        ], name="description_branch")

        # Category embedding branch
        self.category_embedding = tf.keras.Sequential([
            self.category_vectorizer,
            tf.keras.layers.Embedding(
                input_dim=self.category_vectorizer.vocabulary_size(),
                output_dim=32,  # Smaller embedding for categories
                mask_zero=True,
                name="category_embedding"
            ),
            tf.keras.layers.GlobalAveragePooling1D(name="category_pooling"),
        ], name="category_branch")

        # Fusion layer to combine description and category embeddings
        self.fusion_dense = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu", name="fusion_dense_1"),
            tf.keras.layers.BatchNormalization(name="fusion_bn_1"),
            tf.keras.layers.Dropout(0.3, name="fusion_dropout_1"),
            tf.keras.layers.Dense(128, activation="relu", name="fusion_dense_2"),
            tf.keras.layers.BatchNormalization(name="fusion_bn_2"),
            tf.keras.layers.Dropout(0.3, name="fusion_dropout_2"),
            tf.keras.layers.Dense(embedding_dim, name="fusion_output")
        ], name="fusion_layer")

    def call(self, inputs, training=None):
        # Process description
        description_emb = self.description_embedding(inputs["item_description"])

        # Process category
        category_emb = self.category_embedding(inputs["category"])

        # Concatenate description and category embeddings
        combined = tf.concat([description_emb, category_emb], axis=1)

        # Apply fusion layer
        output = self.fusion_dense(combined, training=training)

        return output

class SelfSupervisedItemTwoTower(tfrs.Model):
    def __init__(self, item_model):
        super().__init__()
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval()

    def compute_loss(self, features, training=False):
        item_embeddings = self.item_model(features, training=training)
        return self.task(query_embeddings=item_embeddings, candidate_embeddings=item_embeddings)

# Initialize the enhanced item tower
item_tower = EnhancedItemModel(description_vectorizer, category_vectorizer, embedding_dimension)
item_model_trainer = SelfSupervisedItemTwoTower(item_tower)
item_model_trainer.compile(optimizer=tf.keras.optimizers.Adam(0.001))

# Prepare training data with both description and category
train_item_ds = items_ds.map(lambda x: {
    "item_description": x["item_description"],
    "category": x["category"]
}).batch(512).cache()

print("Training Enhanced Item Tower with description and category...")
item_model_trainer.fit(train_item_ds, epochs=15, verbose=1)
print("Enhanced Item Tower training complete.")

# --- Step 3: Generate and Store Item Embeddings in Faiss ---
print("\n[3] Generating item embeddings and storing in Faiss...")
index = faiss.IndexFlatL2(embedding_dimension)

# Generate embeddings using both description and category
item_embeddings_generator = items_ds.batch(512).map(lambda x: item_tower({
    "item_description": x["item_description"],
    "category": x["category"]
}))
all_item_embeddings = np.concatenate(list(item_embeddings_generator.as_numpy_iterator()))

# Normalize embeddings for better similarity search
all_item_embeddings = all_item_embeddings / np.linalg.norm(all_item_embeddings, axis=1, keepdims=True)

index.add(all_item_embeddings)
print(f"Faiss index now contains {index.ntotal} vectors.")
index_to_item_id = {i: item_id for i, item_id in enumerate(items_df["item_id"])}

# --- Step 4: Train the User Tower ---
print("\n[4] Building and training the User Tower...")
unique_user_ids = interactions_df["user_id"].unique()

class UserModel(tf.keras.Model):
    def __init__(self, user_ids):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(user_ids) + 1, embedding_dimension)
        ])
        self.dense = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(embedding_dimension)
        ])

    def call(self, inputs):
        return self.dense(self.user_embedding(inputs))

class UserItemRetrievalModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.item_model.trainable = False  # Keep item model frozen

        self.task = tfrs.tasks.Retrieval()

    def compute_loss(self, data, training=False):
        user_embeddings = self.user_model(data["user_id"])

        # Get item embeddings for the interacted items
        item_data = {
            "item_description": data["item_description"],
            "category": data["category"]
        }
        item_embeddings = self.item_model(item_data, training=False)

        # Normalize embeddings
        user_embeddings = tf.nn.l2_normalize(user_embeddings, axis=1)
        item_embeddings = tf.nn.l2_normalize(item_embeddings, axis=1)

        return self.task(
            query_embeddings=user_embeddings,
            candidate_embeddings=item_embeddings
        )

# Prepare training data with item descriptions and categories
interactions_with_details_df = pd.merge(
    interactions_df,
    items_df[['item_id', 'item_description', 'category']],
    on='item_id'
)

# Create negative sampling for better training
print("Creating training dataset with negative sampling...")
positive_interactions = interactions_with_details_df.copy()
positive_interactions['label'] = 1.0

# Create negative samples
negative_interactions = []
for user_id in unique_user_ids:
    user_positive_items = set(positive_interactions[positive_interactions['user_id'] == user_id]['item_id'])

    # Sample negative items (items the user hasn't interacted with)
    num_negatives = min(len(user_positive_items), 20)  # Limit negatives to prevent memory issues
    all_items = set(items_df['item_id'])
    negative_items = list(all_items - user_positive_items)

    if len(negative_items) >= num_negatives:
        sampled_negatives = np.random.choice(negative_items, size=num_negatives, replace=False)

        for item_id in sampled_negatives:
            item_row = items_df[items_df['item_id'] == item_id].iloc[0]
            negative_interactions.append({
                'user_id': user_id,
                'item_id': item_id,
                'item_description': item_row['item_description'],
                'category': item_row['category'],
                'label': 0.0
            })

negative_interactions_df = pd.DataFrame(negative_interactions)
full_training_df = pd.concat([positive_interactions, negative_interactions_df], ignore_index=True)

# Shuffle the training data
full_training_df = full_training_df.sample(frac=1).reset_index(drop=True)

full_interactions_ds = tf.data.Dataset.from_tensor_slices(dict(full_training_df))
train_ds_user = full_interactions_ds.shuffle(50_000).batch(512).cache()

user_tower = UserModel(unique_user_ids)
user_model_trainer = UserItemRetrievalModel(user_tower, item_tower)
user_model_trainer.compile(optimizer=tf.keras.optimizers.Adam(0.001))

# Train the user model
print("Training User Tower...")
user_model_trainer.fit(train_ds_user, epochs=15, verbose=1)
print("User Tower training complete.")

# --- Step 5: Enhanced Recommendation Function ---
print("\n[5] Implementing recommendation functions...")

def get_balanced_recommendations(user_id, top_k=15):
    """Get recommendations with explicit category balancing"""
    print(f"\n--- Balanced recommendations for user '{user_id}' ---")

    if user_id not in unique_user_ids:
        print(f"User '{user_id}' is a new user (cold start).")
        return None

    # Get user's preferred categories
    preferred_categories = user_categories[user_id]
    print(f"User's preferred categories ({len(preferred_categories)}): {list(preferred_categories)}")

    # Get user embedding and normalize it
    user_embedding = user_tower(tf.constant([user_id])).numpy()
    user_embedding = user_embedding / np.linalg.norm(user_embedding, axis=1, keepdims=True)

    # Get items user has already interacted with
    user_interacted_items = set(interactions_df[interactions_df['user_id'] == user_id]['item_id'])

    # Get recommendations for each preferred category
    recommendations = []
    items_per_category = max(1, top_k // len(preferred_categories))

    for i, category in enumerate(preferred_categories):
        # Get items from this specific category
        category_items = items_df[items_df['category'] == category]

        # Filter out already interacted items
        available_category_items = category_items[
            ~category_items['item_id'].isin(user_interacted_items)
        ]

        if len(available_category_items) == 0:
            continue

        # Get embeddings for items in this category
        category_item_indices = []
        for _, item_row in available_category_items.iterrows():
            item_idx = int(item_row['item_id'])
            category_item_indices.append(item_idx)

        # Calculate similarities to items in this category
        category_embeddings = all_item_embeddings[category_item_indices]
        similarities = np.dot(user_embedding, category_embeddings.T).flatten()

        # Get top items from this category
        top_indices = np.argsort(similarities)[::-1][:items_per_category]

        for rank, idx in enumerate(top_indices):
            item_row = available_category_items.iloc[idx]
            similarity = similarities[idx]

            recommendations.append({
                'item_id': item_row['item_id'],
                'category': item_row['category'],
                'title': item_row['item_title'],
                'similarity': similarity,
                'rank_in_category': rank + 1
            })

    # Sort all recommendations by similarity
    recommendations.sort(key=lambda x: x['similarity'], reverse=True)
    recommendations = recommendations[:top_k]

    # Calculate accuracy
    category_counts = defaultdict(int)
    correct_recommendations = 0

    for rec in recommendations:
        category_counts[rec['category']] += 1
        if rec['category'] in preferred_categories:
            correct_recommendations += 1

    accuracy = correct_recommendations / len(recommendations) if recommendations else 0

    print(f"Generated {len(recommendations)} recommendations")
    print(f"Category distribution:")
    for category in preferred_categories:
        count = category_counts[category]
        percentage = (count / len(recommendations)) * 100 if recommendations else 0
        print(f"  ✓ {category}: {count} items ({percentage:.1f}%)")

    print(f"Accuracy: {correct_recommendations}/{len(recommendations)} ({accuracy:.1%})")

    return accuracy

def get_collaborative_recommendations(user_id, top_k=15):
    """Pure collaborative filtering recommendations"""
    print(f"\n--- Collaborative filtering for user '{user_id}' ---")

    if user_id not in unique_user_ids:
        print(f"User '{user_id}' is a new user (cold start).")
        return None

    preferred_categories = user_categories[user_id]
    print(f"User's preferred categories ({len(preferred_categories)}): {list(preferred_categories)}")

    # Get user embedding
    user_embedding = user_tower(tf.constant([user_id])).numpy()
    user_embedding = user_embedding / np.linalg.norm(user_embedding, axis=1, keepdims=True)

    # Get items user has already interacted with
    user_interacted_items = set(interactions_df[interactions_df['user_id'] == user_id]['item_id'])

    # Search for similar items
    search_k = min(top_k * 3, index.ntotal)
    distances, indices = index.search(user_embedding, search_k)

    recommendations = []
    for idx in indices[0]:
        item_id = index_to_item_id[idx]
        if item_id in user_interacted_items:
            continue

        item_row = items_df[items_df['item_id'] == item_id].iloc[0]
        recommendations.append({
            'item_id': item_id,
            'category': item_row['category'],
            'title': item_row['item_title']
        })

        if len(recommendations) >= top_k:
            break

    # Calculate accuracy
    category_counts = defaultdict(int)
    correct_recommendations = 0

    for rec in recommendations:
        category_counts[rec['category']] += 1
        if rec['category'] in preferred_categories:
            correct_recommendations += 1

    accuracy = correct_recommendations / len(recommendations) if recommendations else 0

    print(f"Generated {len(recommendations)} recommendations")
    print(f"Category distribution:")
    for category in sorted(category_counts.keys()):
        count = category_counts[category]
        percentage = (count / len(recommendations)) * 100 if recommendations else 0
        is_preferred = "✓" if category in preferred_categories else "✗"
        print(f"  {is_preferred} {category}: {count} items ({percentage:.1f}%)")

    print(f"Accuracy: {correct_recommendations}/{len(recommendations)} ({accuracy:.1%})")

    return accuracy

# --- Step 6: Testing and Evaluation ---
print("\n[6] Testing recommendation system...")

def comprehensive_test():
    print(f"\n" + "="*80)
    print("COMPREHENSIVE RECOMMENDATION SYSTEM TEST")
    print(f"Categories: {len(categories)}")
    print(f"Users: {num_users}")
    print(f"Items: {num_items}")
    print(f"Total interactions: {len(interactions_df)}")
    print("="*80)

    # Test multiple users
    test_users = np.random.choice(unique_user_ids, size=10, replace=False)

    balanced_accuracies = []
    collaborative_accuracies = []

    for i, user_id in enumerate(test_users):
        print(f"\n{'='*60}")
        print(f"TEST {i+1}/10: User ID '{user_id}'")
        print(f"{'='*60}")

        # Show user stats
        user_interactions = interactions_df[interactions_df['user_id'] == user_id]
        preferred_categories = user_categories[user_id]

        print(f"User profile:")
        print(f"  - Preferred categories: {list(preferred_categories)}")
        print(f"  - Total interactions: {len(user_interactions)}")

        # Test balanced approach
        balanced_acc = get_balanced_recommendations(user_id, top_k=12)
        if balanced_acc is not None:
            balanced_accuracies.append(balanced_acc)

        # Test collaborative approach
        collaborative_acc = get_collaborative_recommendations(user_id, top_k=12)
        if collaborative_acc is not None:
            collaborative_accuracies.append(collaborative_acc)

    # Overall results
    print(f"\n" + "="*80)
    print("OVERALL RESULTS")
    print("="*80)

    if balanced_accuracies:
        avg_balanced = np.mean(balanced_accuracies)
        std_balanced = np.std(balanced_accuracies)
        print(f"Balanced Approach:")
        print(f"  Average accuracy: {avg_balanced:.1%} ± {std_balanced:.1%}")
        print(f"  Range: {min(balanced_accuracies):.1%} - {max(balanced_accuracies):.1%}")

    if collaborative_accuracies:
        avg_collaborative = np.mean(collaborative_accuracies)
        std_collaborative = np.std(collaborative_accuracies)
        print(f"Collaborative Filtering:")
        print(f"  Average accuracy: {avg_collaborative:.1%} ± {std_collaborative:.1%}")
        print(f"  Range: {min(collaborative_accuracies):.1%} - {max(collaborative_accuracies):.1%}")

    # Expected random performance
    avg_categories_per_user = np.mean([len(cats) for cats in user_categories.values()])
    random_accuracy = avg_categories_per_user / len(categories)
    print(f"Expected random accuracy: {random_accuracy:.1%}")

    # Performance comparison
    if balanced_accuracies and collaborative_accuracies:
        if avg_balanced > avg_collaborative:
            improvement = ((avg_balanced - avg_collaborative) / avg_collaborative) * 100
            print(f"✓ Balanced approach performs {improvement:.1f}% better")
        else:
            improvement = ((avg_collaborative - avg_balanced) / avg_balanced) * 100
            print(f"✓ Collaborative approach performs {improvement:.1f}% better")

    print("\n" + "="*80)
    print("SYSTEM ANALYSIS COMPLETE")
    print("="*80)

# Run comprehensive test
comprehensive_test()

# Quick analysis of category distribution
print(f"\n" + "="*60)
print("CATEGORY ANALYSIS")
print("="*60)

category_stats = []
for category in categories:
    items_count = len(category_to_items[category])
    interactions_count = len(interactions_with_details_df[interactions_with_details_df['category'] == category])
    users_count = len(set(interactions_with_details_df[interactions_with_details_df['category'] == category]['user_id']))

    category_stats.append({
        'category': category,
        'items': items_count,
        'interactions': interactions_count,
        'users': users_count
    })

category_stats.sort(key=lambda x: x['interactions'], reverse=True)

print("Category popularity (by interactions):")
for i, stats in enumerate(category_stats[:10]):
    print(f"  {i+1:2d}. {stats['category']:15s}: {stats['interactions']:4d} interactions, {stats['users']:3d} users")

print(f"\nLeast popular categories:")
for i, stats in enumerate(category_stats[-5:]):
    print(f"  {stats['category']:15s}: {stats['interactions']:4d} interactions, {stats['users']:3d} users")

[1] Fabricating data with 20 categories and diverse user preferences...
Generating user interactions with diverse category preferences...
Generated 10000 items across 20 categories
Generated 11829 interactions from 1000 users
Average interactions per user: 11.8

User category distribution:
  3 categories: 327 users (32.7%)
  4 categories: 373 users (37.3%)
  5 categories: 300 users (30.0%)

Top 10 most interacted categories:
  1. kitchen_dining: 660 interactions
  2. books: 652 interactions
  3. watches: 649 interactions
  4. clothing: 608 interactions
  5. sports_outdoors: 608 interactions
  6. office_supplies: 605 interactions
  7. music: 605 interactions
  8. home_garden: 604 interactions
  9. industrial: 603 interactions
  10. automotive: 598 interactions

Sample user category preferences:
User 0: [np.str_('office_supplies'), np.str_('kitchen_dining'), np.str_('jewelry'), np.str_('grocery'), np.str_('toys_games')] (18 total interactions)
User 1: [np.str_('clothing'), np.str_('groce